In [ ]:
# all data sourced from https://www.kaggle.com/hm-land-registry/uk-housing-prices-paid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import linear_model
from sklearn.svm import SVR
""" important note conveyancy takes 6-8 weeks - this means that from accepting offer to 
Date of Transfer could be two months different - this will be accounted for later in document"""

In [ ]:
# Load data
housing_df = pd.read_csv('price_paid_records.csv')

In [ ]:
# Display head of data
housing_df.head()

In [ ]:
#D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other 
def property_filter_out(data,col,filter_out=''):
    for item in filter_out:
        data = data[data[col]!=item]
    return data

In [ ]:
# Convert the date column to actual datetime + create month/year columns
housing_df['Date of Transfer'] = pd.to_datetime(housing_df['Date of Transfer'],dayfirst=True)
housing_df['Year'] = housing_df['Date of Transfer'].dt.year
housing_df['Month'] = housing_df['Date of Transfer'].dt.month
# filter out data we don't want
housing_df = property_filter_out(housing_df,'Property Type','TFO')

In [ ]:
# Create a pivot table of data and select only the last 10 years
housing_pivot = pd.pivot_table(housing_df,values='Price',index='Month',columns='Year',aggfunc='mean')
time_range = np.arange(2006,2017)
housing_pivot = housing_pivot[time_range]

In [ ]:
# Plot data
plt.figure(figsize=(10,5))

plt.subplot(1,1,1)
plt.plot(housing_pivot)

plt.legend(time_range,loc=4)
plt.xlabel('Month Jan-Dec')
plt.ylabel("Price - pounds")
plt.title("Price vs Month")
plt.show()

In [ ]:
# Take the mean across years for each month - does price really vary by month?
avg_across_years = np.empty((12,1))

for i in range(0,12):
    avg_across_years[i,0] = np.mean(np.array(housing_pivot)[i,:])
plt.plot(avg_across_years)
plt.ylabel('Price - Pounds')
plt.xlabel('Month Jan - Dec')
plt.show()

In [ ]:
# What the above appears to show is that house prices vary by time of year
# If it takes 2 months from acceptance of an offer to completion of handover
# then lowest price point in the market will be the Winter months (Dec-Feb)
# Does this hold true in all cases? Is this different by region?
# Could this difference relate to weather? 
#(will need average monthly temp by year for each region)
# Can we use:
# https://www.metoffice.gov.uk/public/weather/climate-historic/#?tab=climateHistoric
# Would this explain the variance from the best fit?

In [ ]:
# All regions:
print(pd.unique(housing_df['County']))

In [ ]:
def region_plot(county,year_start=2015,year_end=2017):
    housing_pivot = pd.pivot_table(housing_df[housing_df['County']==county],values='Price',index='Month',columns='Year',aggfunc='mean')
    time_range = np.arange(year_start,year_end)
    housing_pivot = housing_pivot[time_range]
    X=np.array(housing_pivot.index)
    for year in time_range:
        y=housing_pivot[year]
        housing_pivot['{0}_BestFit'.format(year)]=best_fit(X,y)
    # Plot data
    plt.figure(figsize=(10,5))
    plt.subplot(1,1,1)
    plt.plot(housing_pivot)
    plt.legend(housing_pivot.columns,loc=4)
    plt.xlabel('Month Jan-Dec')
    plt.ylabel("Price - pounds")
    plt.title("Price vs Month")
    plt.show()
    return housing_pivot

In [ ]:
piv=region_plot('GREATER LONDON',year_start=2012)

In [ ]:
def best_fit(X,y):
    lr = linear_model.LinearRegression()
    X=X.reshape(len(X),1)
    y=y.reshape(len(y),1)
    lr.fit(X,y)
    return lr.predict(X)

In [ ]:
# we have data from UK as a whole 
uk_temp_df = pd.read_excel('weather1.xlsx')

In [ ]:
housing_pivot = pd.pivot_table(housing_df,values='Price',index='Month',columns='Year',aggfunc='mean')
time_range = np.arange(2013,2017)
housing_pivot = housing_pivot[time_range]

In [ ]:
housing_pivot

In [ ]:
lr = linear_model.LinearRegression()
svr = SVR(kernel='linear')

In [ ]:
first = True
for year in time_range:
    temp_arr = np.ones((12,5))
    temp_arr[:,0] = np.array([datetime.date(year,int(m),1) for m in housing_pivot.index],dtype='datetime64')
    #weather squared
    temp_arr[:,2] = np.array(uk_temp_df[str(year)])**2
    # weather cubed
    temp_arr[:,3] = np.array(uk_temp_df[str(year)])**3
    temp_arr[:,4] = np.array(housing_pivot[year])
    #print(np.array(housing_pivot[year]))
    #print(uk_temp_df[str(year)])
    if first == True:
        X = temp_arr
        first = False
    else:
        X = np.concatenate((X,temp_arr),axis=0)

In [ ]:
# fit linear model
# fit simply with just datetime
y = X[:,4]
lr.fit(X[:,:1],y)
lr_pred1 = lr.predict(X[:,:1])
# fit with temp
lr.fit(X[:,:2],y)
lr_pred2 = lr.predict(X[:,:2])
# with with temp squared
lr.fit(X[:,:3],y)
lr_pred3 = lr.predict(X[:,:3])
#fit with temp cubed
lr.fit(X[:,:4],y)
lr_pred4 = lr.predict(X[:,:4])

In [ ]:
y

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(X[:,0],y)
plt.plot(X[:,0],y,X[:,0],lr_pred1,X[:,0],lr_pred2,X[:,0],lr_pred3,X[:,0],lr_pred4)
#plt.legend(['Price data','Regression','Regression with temp',
#            'Regression with squared temp','Regression with cubed temp'])
plt.show()


In [ ]:
import datetime

In [ ]:
datetime.date(2010,2,1)

In [ ]:
plt.subplot(1,2,1)
plt.plot(X[:,0],lr_pred4)
plt.subplot(1,2,2)
plt.plot(X[:,0],y)

In [ ]:
lr_pred4-y

In [ ]:
np.array(lr.coef_,dtype='int')